In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import IsolationForest
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)

In [ ]:
df = pd.read_csv("final_anomalouslogin_modified3.csv") # Read CSV
df

,company_id,user_id,timestamp_utc,ip_address,device_id,os,browser,login_success,asn,ip_reputation,ip_vpn_tor,num_distinct_devices_last30d,failed_attempts_last_10m,hours_since_prev_login,distance_from_prev_login_km,speed_kmh,dist_from_user_login,prediction,risk_score,reasons
0,instabook,user0126,2025-08-27 15:43:00,106.221.201.140,53f5129f-940b-4bac-81ca-0912b76515e3,Android,Chrome,0,AS13335,0,0,3,2,38.89,8.452,0.22,4.077,0,0,Normal: baseline behaviour (risk 0)
1,facegram,user0249,2025-08-15 10:29:00,115.98.63.211,ef06e5bd-98d8-4e21-a79e-6a9c78de0972,Android,Chrome,0,AS174,0,0,2,3,15.44,12.796,0.82,43.170,0,25,Sudden OS/Browser change during failed attempts
2,instabook,user0046,2025-08-26 16:49:00,182.72.95.90,f3ad84fb-6982-4b78-8756-220f45c0ab03,macOS,Firefox,1,AS13335,2,0,4,0,13.00,30.834,2.35,12.810,0,25,Suspicious IP / VPN detected
3,instabook,user0252,2025-08-25 20:05:00,182.68.177.35,99888ec1-a2de-4269-9d82-539ea895c5e2,Windows 10,Edge,0,AS55836,2,0,4,2,9.02,29.218,3.20,2.885,0,50,Suspicious IP / VPN detected; Sudden OS/Browse...
4,instabook,user0144,2025-08-11 20:49:00,49.37.239.250,80c6bded-1314-4a8c-870e-21a1c2a1477e,macOS,Chrome,0,AS3356,2,0,2,2,29.23,25.531,0.87,39.285,0,50,Suspicious IP / VPN detected; Sudden OS/Browse...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,facegram,user0256,2025-08-23 12:50:00,49.38.0.101,c5d9deb1-d52b-4b23-9c87-efffe1b2440d,macOS,Safari,1,AS9498,1,0,5,0,31.00,32.440,1.04,42.831,0,25,Suspicious IP / VPN detected
9996,facegram,user0390,2025-08-01 07:40:00,182.75.254.109,9fca5b76-d45b-4f18-b336-251f24a15c92,Windows 11,Safari,0,AS174,0,0,4,3,9.04,27.401,3.00,4.583,0,0,Normal: baseline behaviour (risk 0)
9997,instabook,user0347,2025-08-24 14:32:00,115.102.127.87,7b35ad57-e5d1-45d3-82ef-63eff964f67f,Windows 10,Safari,0,AS3356,0,0,1,1,38.80,29.991,0.77,15.650,0,25,Sudden OS/Browser change during failed attempts
9998,instabook,user0208,2025-08-03 16:45:00,182.71.12.173,79134ed5-8bac-490e-bc6f-a6cdad94600e,macOS,Safari,0,AS55836,1,0,4,3,12.91,5.520,0.42,23.354,0,25,Suspicious IP / VPN detected


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   company_id                    10000 non-null  object 
 1   user_id                       10000 non-null  object 
 2   timestamp_utc                 10000 non-null  object 
 3   ip_address                    10000 non-null  object 
 4   device_id                     10000 non-null  object 
 5   os                            10000 non-null  object 
 6   browser                       10000 non-null  object 
 7   login_success                 10000 non-null  int64  
 8   asn                           10000 non-null  object 
 9   ip_reputation                 10000 non-null  int64  
 10  ip_vpn_tor                    10000 non-null  int64  
 11  num_distinct_devices_last30d  10000 non-null  int64  
 12  failed_attempts_last_10m      10000 non-null  int64  
 13  ho

In [ ]:
df.describe()

,login_success,ip_reputation,ip_vpn_tor,num_distinct_devices_last30d,failed_attempts_last_10m,hours_since_prev_login,distance_from_prev_login_km,speed_kmh,dist_from_user_login,prediction,risk_score
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,0.226600,1.021600,0.00830,3.545100,1.606900,24.415985,60.774653,5.47103,82.299246,0.091700,39.350000
std,0.418653,0.817802,0.09073,1.797634,1.404055,13.525921,269.876564,42.19270,428.418917,0.288616,19.258104
min,0.000000,0.000000,0.00000,1.000000,0.000000,1.000000,0.000000,0.00000,0.009000,0.000000,0.000000
25%,0.000000,0.000000,0.00000,2.000000,0.000000,12.700000,12.311250,0.52000,12.631000,0.000000,25.000000
50%,0.000000,1.000000,0.00000,4.000000,2.000000,24.430000,25.692000,1.03000,25.489000,0.000000,45.000000
75%,0.000000,2.000000,0.00000,5.000000,3.000000,35.950000,38.406250,2.06000,38.297250,0.000000,50.000000
max,1.000000,2.000000,1.00000,10.000000,10.000000,48.000000,2993.414000,1661.75000,4961.555000,1.000000,100.000000


In [ ]:
df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"])

# Feature engineering
df["login_hour"] = df["timestamp_utc"].dt.hour
df["day_of_week"] = df["timestamp_utc"].dt.dayofweek
df["is_weekend"] = df["day_of_week"].isin([5,6]).astype(int)

In [ ]:
y = df["prediction"]
y

,prediction
0,0
1,0
2,0
3,0
4,0
...,...
9995,0
9996,0
9997,0
9998,0


In [ ]:
categorical_cols = ["company_id", "user_id", "device_id", "asn",
                    "ip_address", "os", "browser"]
# Label Encoding
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

In [ ]:
features = categorical_cols + [
    "num_distinct_devices_last30d",
    "failed_attempts_last_10m",
    "hours_since_prev_login",
    "distance_from_prev_login_km",
    "speed_kmh",
    "dist_from_user_login",
    "ip_reputation",
    "ip_vpn_tor",
    "login_success",
    "login_hour",
    "is_weekend"
]
X = df[features]
X

,company_id,user_id,device_id,asn,ip_address,os,browser,num_distinct_devices_last30d,failed_attempts_last_10m,hours_since_prev_login,distance_from_prev_login_km,speed_kmh,dist_from_user_login,ip_reputation,ip_vpn_tor,login_success,login_hour,is_weekend
0,1,125,3290,0,923,0,0,3,2,38.89,8.452,0.22,4.077,0,0,0,15,0
1,0,248,9322,1,3988,0,0,2,3,15.44,12.796,0.82,43.170,0,0,0,10,0
2,1,45,9499,0,7134,5,2,4,0,13.00,30.834,2.35,12.810,2,0,1,16,0
3,1,251,5999,3,6561,2,1,4,2,9.02,29.218,3.20,2.885,2,0,0,20,0
4,1,143,5050,2,8373,5,0,2,2,29.23,25.531,0.87,39.285,2,0,0,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,255,7704,4,8403,5,3,5,0,31.00,32.440,1.04,42.831,1,0,1,12,1
9996,0,389,6240,1,7457,3,3,4,3,9.04,27.401,3.00,4.583,0,0,0,7,0
9997,1,346,4831,2,2161,2,3,1,1,38.80,29.991,0.77,15.650,0,0,0,14,1
9998,1,207,4735,3,6903,5,3,4,3,12.91,5.520,0.42,23.354,1,0,0,16,1


In [ ]:
# Time-based split

df_sorted = df.sort_values("timestamp_utc")
split_idx = int(0.7 * len(df_sorted)) #Chooses an index at 70% of the data for train/test split (70% train, 30% test).
X_train, X_test = df_sorted[features].iloc[:split_idx], df_sorted[features].iloc[split_idx:]
y_train, y_test = df_sorted["prediction"].iloc[:split_idx], df_sorted["prediction"].iloc[split_idx:]

# Scale numeric features for Isolation Forest
scaler = StandardScaler() # IsolationForest is sensitive to feature scales, so scaling is applied.
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
numeric_cols = ["hours_since_prev_login", "distance_from_prev_login_km", "speed_kmh",
                "dist_from_user_login", "ip_reputation", "failed_attempts_last_10m"]

# Adding Noise for reducing accuracy
X_train_noisy = X_train.copy()
for col in numeric_cols:
    X_train_noisy[col] = X_train_noisy[col] * np.random.uniform(0.9, 1.1, size=len(X_train_noisy))

# Optional: flip small portion of labels

y_train_noisy = y_train.copy()
flip_idx = np.random.choice(y_train.index, size=int(0.05*len(y_train)), replace=False)
y_train_noisy.loc[flip_idx] = 1 - y_train_noisy.loc[flip_idx]

In [ ]:
# Train Models
# XGBoostClassifier

xgb_clf = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric="logloss",
    max_depth=2,
    n_estimators=30,
    learning_rate=0.2,
    random_state=42
)
xgb_clf.fit(X_train_noisy, y_train_noisy)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [08:30:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=30, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# Isolation Forest

iso = IsolationForest(contamination=0.1, random_state=42)
iso.fit(X_train_scaled)

IsolationForest(contamination=0.1, random_state=42)

In [ ]:
# Risk Score Calculation

# XGBoost probabilities
prob_xgb = xgb_clf.predict_proba(X_test)[:,1]

# Isolation Forest anomaly scores
iso_scores = -iso.decision_function(X_test_scaled)   # higher = anomaly
# Normalize to 0–1
prob_iso = (iso_scores - iso_scores.min()) / (iso_scores.max() - iso_scores.min() + 1e-9)

# Weighted ensemble risk score
risk_scores = (0.5 * prob_xgb + 0.5 * prob_iso) * 100

# Final predictions
y_pred = (risk_scores >= 60).astype(int)

In [ ]:
# Reasons Function
def get_reasons(row):
    reasons = []

    if row["speed_kmh"] > 1000:
        reasons.append(f"Impossible travel detected")

    if row["ip_vpn_tor"] == 1:
        reasons.append("VPN/Tor detected")
    if row["ip_reputation"] > 7:
        reasons.append("High-risk IP reputation")

    if row["failed_attempts_last_10m"] > 5 and row["login_success"] == 0:
        reasons.append("Multiple failed logins in short time")

    if row["login_hour"] < 6 or row["login_hour"] > 22:
        reasons.append(f"Login at unusual hour: {row['login_hour']}")

    return "; ".join(reasons) if reasons else "Normal baseline behaviour"

In [ ]:
# Results with Reasons

X_test_df = pd.DataFrame(X_test, columns=features)
results = pd.DataFrame({
    "timestamp": df_sorted["timestamp_utc"].iloc[split_idx:].values,
    "actual": y_test.values,
    "prediction": y_pred,
    "risk_score": np.round(risk_scores, 2)
})
results["reasons"] = X_test_df.apply(get_reasons, axis=1)

In [ ]:
# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

precision = precision_score(y_test, y_pred)
print(f"Precision: {precision*100:.2f}%")

recall = recall_score(y_test, y_pred)
print(f"Recall: {recall*100:.2f}%")

f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1*100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Normal", "Anomalous"]))

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix (Counts):")
print(cm)

cm_percent = cm.astype("float") / cm.sum() * 100
print("\nConfusion Matrix (% of total, 0–100 scale):")
print(np.round(cm_percent, 2))

roc_auc = roc_auc_score(y_test, risk_scores/100)
print("\nROC-AUC Score:", round(roc_auc, 4))

Accuracy: 92.20%
Precision: 94.20%
Recall: 22.03%
F1 Score: 35.71%

Classification Report:
              precision    recall  f1-score   support

      Normal       0.92      1.00      0.96      2705
   Anomalous       0.94      0.22      0.36       295

    accuracy                           0.92      3000
   macro avg       0.93      0.61      0.66      3000
weighted avg       0.92      0.92      0.90      3000


Confusion Matrix (Counts):
[[2701    4]
 [ 230   65]]

Confusion Matrix (% of total, 0–100 scale):
[[90.03  0.13]
 [ 7.67  2.17]]

ROC-AUC Score: 0.997


In [ ]:
# ================================
# Fresh Login Test Cases (Expanded 12 logins for new dataset)
# ================================
fresh_logins = pd.DataFrame([
    # 1. Normal login (baseline)
    {"company_id":1, "user_id":"user_hash_6001", "device_id":"dev_hash_10001", "asn":2101, "ip_address":"ip_hash_3101",
     "os":1, "browser":1, "num_distinct_devices_last30d":1, "failed_attempts_last_10m":0,
     "hours_since_prev_login":24, "distance_from_prev_login_km":3, "speed_kmh":5,
     "dist_from_user_login":2, "ip_reputation":1, "ip_vpn_tor":0, "login_success":1,
     "login_hour":10, "is_weekend":0},

    # 2. Normal login (different company/user)
    {"company_id":2, "user_id":"user_hash_6002", "device_id":"dev_hash_10002", "asn":2102, "ip_address":"ip_hash_3102",
     "os":2, "browser":2, "num_distinct_devices_last30d":2, "failed_attempts_last_10m":1,
     "hours_since_prev_login":12, "distance_from_prev_login_km":10, "speed_kmh":8,
     "dist_from_user_login":5, "ip_reputation":2, "ip_vpn_tor":0, "login_success":1,
     "login_hour":14, "is_weekend":1},

    # 3. Impossible travel anomaly
    {"company_id":1, "user_id":"user_hash_6003", "device_id":"dev_hash_10003", "asn":2103, "ip_address":"ip_hash_3103",
     "os":1, "browser":1, "num_distinct_devices_last30d":3, "failed_attempts_last_10m":0,
     "hours_since_prev_login":1, "distance_from_prev_login_km":9000, "speed_kmh":4500,
     "dist_from_user_login":9000, "ip_reputation":2, "ip_vpn_tor":0, "login_success":1,
     "login_hour":2, "is_weekend":0},

    # 4. VPN + failed logins + high-risk IP
    {"company_id":2, "user_id":"user_hash_6004", "device_id":"dev_hash_10004", "asn":2104, "ip_address":"ip_hash_3104",
     "os":2, "browser":2, "num_distinct_devices_last30d":2, "failed_attempts_last_10m":8,
     "hours_since_prev_login":1, "distance_from_prev_login_km":20, "speed_kmh":5,
     "dist_from_user_login":20, "ip_reputation":9, "ip_vpn_tor":1, "login_success":0,
     "login_hour":23, "is_weekend":1},

    # 5. Unusual hours
    {"company_id":1, "user_id":"user_hash_6005", "device_id":"dev_hash_10005", "asn":2105, "ip_address":"ip_hash_3105",
     "os":1, "browser":2, "num_distinct_devices_last30d":1, "failed_attempts_last_10m":0,
     "hours_since_prev_login":10, "distance_from_prev_login_km":5, "speed_kmh":7,
     "dist_from_user_login":5, "ip_reputation":3, "ip_vpn_tor":0, "login_success":1,
     "login_hour":0, "is_weekend":0},

    # 6. Mixed anomalies
    {"company_id":2, "user_id":"user_hash_6006", "device_id":"dev_hash_10006", "asn":2106, "ip_address":"ip_hash_3106",
     "os":2, "browser":1, "num_distinct_devices_last30d":4, "failed_attempts_last_10m":6,
     "hours_since_prev_login":3, "distance_from_prev_login_km":100, "speed_kmh":1200,
     "dist_from_user_login":100, "ip_reputation":8, "ip_vpn_tor":1, "login_success":0,
     "login_hour":1, "is_weekend":1},

    # 7. Normal login
    {"company_id":1, "user_id":"user_hash_6007", "device_id":"dev_hash_10007", "asn":2107, "ip_address":"ip_hash_3107",
     "os":1, "browser":1, "num_distinct_devices_last30d":1, "failed_attempts_last_10m":0,
     "hours_since_prev_login":20, "distance_from_prev_login_km":3, "speed_kmh":4,
     "dist_from_user_login":3, "ip_reputation":1, "ip_vpn_tor":0, "login_success":1,
     "login_hour":11, "is_weekend":0},

    # 8. Normal login
    {"company_id":2, "user_id":"user_hash_6008", "device_id":"dev_hash_10008", "asn":2108, "ip_address":"ip_hash_3108",
     "os":2, "browser":2, "num_distinct_devices_last30d":2, "failed_attempts_last_10m":0,
     "hours_since_prev_login":18, "distance_from_prev_login_km":10, "speed_kmh":6,
     "dist_from_user_login":8, "ip_reputation":2, "ip_vpn_tor":0, "login_success":1,
     "login_hour":15, "is_weekend":1},

    # 9. Impossible travel anomaly
    {"company_id":1, "user_id":"user_hash_6009", "device_id":"dev_hash_10009", "asn":2109, "ip_address":"ip_hash_3109",
     "os":1, "browser":2, "num_distinct_devices_last30d":2, "failed_attempts_last_10m":7,
     "hours_since_prev_login":2, "distance_from_prev_login_km":500, "speed_kmh":1500,
     "dist_from_user_login":500, "ip_reputation":7, "ip_vpn_tor":1, "login_success":0,
     "login_hour":3, "is_weekend":0},

    # 10. Normal login
    {"company_id":2, "user_id":"user_hash_6010", "device_id":"dev_hash_10010", "asn":2110, "ip_address":"ip_hash_3110",
     "os":2, "browser":1, "num_distinct_devices_last30d":3, "failed_attempts_last_10m":0,
     "hours_since_prev_login":12, "distance_from_prev_login_km":10, "speed_kmh":5,
     "dist_from_user_login":10, "ip_reputation":2, "ip_vpn_tor":0, "login_success":1,
     "login_hour":22, "is_weekend":1},

    # 11. Normal login
    {"company_id":1, "user_id":"user_hash_6011", "device_id":"dev_hash_10011", "asn":2111, "ip_address":"ip_hash_3111",
     "os":1, "browser":2, "num_distinct_devices_last30d":1, "failed_attempts_last_10m":0,
     "hours_since_prev_login":24, "distance_from_prev_login_km":3, "speed_kmh":4,
     "dist_from_user_login":3, "ip_reputation":1, "ip_vpn_tor":0, "login_success":1,
     "login_hour":9, "is_weekend":0},

    # 12. Mixed anomaly
    {"company_id":2, "user_id":"user_hash_6012", "device_id":"dev_hash_10012", "asn":2112, "ip_address":"ip_hash_3112",
     "os":2, "browser":1, "num_distinct_devices_last30d":2, "failed_attempts_last_10m":5,
     "hours_since_prev_login":1, "distance_from_prev_login_km":200, "speed_kmh":500,
     "dist_from_user_login":200, "ip_reputation":6, "ip_vpn_tor":1, "login_success":0,
     "login_hour":1, "is_weekend":0},
])

# ================================
# Encode categoricals (hashed values)
# ================================
for col in categorical_cols:
    if col in fresh_logins.columns:
        le = encoders[col]
        fresh_logins[col] = fresh_logins[col].map(
            lambda x: le.transform([str(x)])[0] if str(x) in le.classes_ else -1
        )

# Scale numeric features
fresh_scaled = scaler.transform(fresh_logins)

# Predictions
prob_xgb_fresh = xgb_clf.predict_proba(fresh_logins)[:,1]
iso_scores_fresh = -iso.decision_function(fresh_scaled)
prob_iso_fresh = (iso_scores_fresh - iso_scores_fresh.min()) / (iso_scores_fresh.max() - iso_scores_fresh.min() + 1e-9)
risk_scores_fresh = (0.5 * prob_xgb_fresh + 0.5 * prob_iso_fresh) * 100

y_pred_fresh = (risk_scores_fresh >= 60).astype(int)

# Apply reasons
fresh_logins["risk_score"] = np.round(risk_scores_fresh, 0)
fresh_logins["prediction"] = y_pred_fresh
fresh_logins["reasons"] = fresh_logins.apply(get_reasons, axis=1)

# Show results
print(fresh_logins[["risk_score", "prediction", "reasons"]])


    risk_score  prediction                                            reasons
0          4.0           0                          Normal baseline behaviour
1         12.0           0                          Normal baseline behaviour
2         56.0           0  Impossible travel detected; Login at unusual h...
3         41.0           0  VPN/Tor detected; High-risk IP reputation; Mul...
4         16.0           0                         Login at unusual hour: 0.0
5         73.0           1  Impossible travel detected; VPN/Tor detected; ...
6          3.0           0                          Normal baseline behaviour
7         11.0           0                          Normal baseline behaviour
8         78.0           1  Impossible travel detected; VPN/Tor detected; ...
9         11.0           0                          Normal baseline behaviour
10         3.0           0                          Normal baseline behaviour
11        72.0           1       VPN/Tor detected; Login at unus